In [1]:
import xgboost, pandas as pd
import duckdb as db 
from xgboost import plot_importance, plot_tree
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.metrics import mean_squared_log_error
import numpy as np 

def calculate_rmsle(y_true, y_pred):
    if (y_true < 0).any() or (y_pred < 0).any():
        raise ValueError("RMSLE cannot be computed when true or predicted values are negative.")
    
    # Calculate the Mean Squared Logarithmic Error (MSLE)
    msle = mean_squared_log_error(y_true, y_pred)
    
    # Calculate the Root Mean Squared Logarithmic Error (RMSLE)
    rmsle = np.sqrt(msle)
    
    return rmsle

In [3]:
df_holidays, df_stores, df_transactions, df_oil = pd.read_csv('data/holidays_events.csv'), pd.read_csv('data/stores.csv'), pd.read_csv('data/transactions.csv'), pd.read_csv('data/oil.csv')

In [6]:
df_final_hols_count = db.sql("""
with cte as (
  select * , month(date::datetime) "month", year(date::datetime) "year", week(date::datetime) "week" from df_holidays
) , hols_count as (
select distinct year, week, count(type) OVER (partition by year, week ORDER BY week ASC) "number_of_holidays" from cte 
), final_hols_count as (
select  distinct year, week, avg(number_of_holidays ) OVER (PARTITION BY year, week) "number_of_holidays" FROM hols_count 
) 
select * from final_hols_count
""").df()

In [7]:
df_transactions = pd.read_csv('data/transactions.csv')
df_transactions = db.sql("""
    select *, avg(transactions) OVER (partition by store_nbr order by date ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING) "7d_transactions_avg" from df_transactions 
""").to_df()
df_transactions

,date,store_nbr,transactions,7d_transactions_avg
0,2013-01-02,11,3547,NaN
1,2013-01-03,11,2675,3547.000000
2,2013-01-04,11,2515,3111.000000
3,2013-01-05,11,3052,2912.333333
4,2013-01-06,11,3188,2947.250000
...,...,...,...,...
83483,2017-08-11,41,1316,1178.000000
83484,2017-08-12,41,1254,1206.500000
83485,2017-08-13,41,1346,1223.625000
83486,2017-08-14,41,1045,1222.500000


In [8]:
df_oil = pd.read_csv('data/oil.csv')
df_oil['date'] = pd.to_datetime(df_oil['date'])
df_oil.set_index('date', inplace=True)
# Create a date range that covers the entire period of the dataset
date_range = pd.date_range(start=df_oil.index.min(), end=df_oil.index.max(), freq='D')
df_oil = df_oil.reindex(date_range, method='ffill')
df_oil['dcoilwtico']= df_oil['dcoilwtico'].bfill()
df_oil.reset_index(inplace=True)
df_oil.rename(columns={'index':'date'}, inplace=True)
df_oil = db.sql("""
    select *, avg(dcoilwtico) OVER (ORDER BY date ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING) "7d_avg_oil_price"
    , avg(dcoilwtico) OVER (ORDER BY date ROWS BETWEEN 16 PRECEDING AND 1 PRECEDING) "15d_avg_oil_price"
    , avg(dcoilwtico) OVER (ORDER BY date ROWS BETWEEN 91 PRECEDING AND 1 PRECEDING) "90d_avg_oil_price" 
    from df_oil 
""").df().fillna(0)
df_oil = db.sql("""
  select *, coalesce(lag(dcoilwtico, 364) OVER (order by date),0) "oil_lag364"
       from df_oil order  by date
""").df()
df_oil

,date,dcoilwtico,7d_avg_oil_price,15d_avg_oil_price,90d_avg_oil_price,oil_lag364
0,2013-01-01,93.14,0.000000,0.000000,0.000000,0.00
1,2013-01-02,93.14,93.140000,93.140000,93.140000,0.00
2,2013-01-03,92.97,93.140000,93.140000,93.140000,0.00
3,2013-01-04,93.12,93.083333,93.083333,93.083333,0.00
4,2013-01-05,93.12,93.092500,93.092500,93.092500,0.00
...,...,...,...,...,...,...
1699,2017-08-27,47.65,47.901250,47.953750,46.751429,47.64
1700,2017-08-28,46.40,47.783750,47.881250,46.730220,46.97
1701,2017-08-29,46.46,47.510000,47.730625,46.694725,46.32
1702,2017-08-30,45.96,47.393750,47.583750,46.659890,44.68


In [27]:
def prep_train_data(df):
    df = df.sort_values(['id'])
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['day_of_week'] = df['date'].dt.day_of_week
    df['day_of_month'] = df['date'].dt.day
    df["quarter"] = df['date'].dt.quarter
    df["weekofyear"] = df['date'].dt.isocalendar().week 
    df['payday'] = df['date'].apply(lambda x: 1 if (pd.to_datetime(x).day==15 or pd.to_datetime(x).is_month_end) else 0)
    df["is_weekend"] = df['day_of_week'].apply(lambda x: 1 if (x in (4,5,6)) else 0) 
    #df_train['day_name'] = df_train['date'].dt.day_name()
    df = db.sql("""   
        select distinct df.*  exclude sales
        , case when (h.type='Event' and description like 'Terremoto Manabi%' and family='BEVERAGES') then 3115
        when (h.type='Event' and description like 'Terremoto Manabi%' and family='GROCERY I') then 4325 
        when (h.type='Event' and description like 'Terremoto Manabi%' and family='CLEANING') then 1150 
        else sales 
        end "sales"  
        , s.cluster , s.type, s.state , s.city 
        , case when h.transferred='False' THEN  COALESCE(h.type, 'Work Day') ELSE 'Work Day' END "holiday_type"
        , o.dcoilwtico "oil_price",    "7d_avg_oil_price",	"90d_avg_oil_price", "15d_avg_oil_price", oil_lag364
        , t."7d_transactions_avg"
        , "number_of_holidays" 
        , case when h.locale = 'National' THEN 1 ELSE 0 END "is_national_holiday" 
        , case when (h.type='Event' and description like 'Terremoto Manabi%') THEN '1' ELSE 0 END "earthquake_impact"    
        from df  
        left join df_transactions t on df.date = t.date and df.store_nbr = t.store_nbr
        left join df_holidays h on h.date = df.date
        left join df_final_hols_count on df.weekofyear = df_final_hols_count.week and df.year=df_final_hols_count.year
        left join df_oil o on o.date = df.date                
        left join df_stores s on s.store_nbr = df.store_nbr 
        order by id 
    """).df()
    #df_train['oil_price'] = df_train['oil_price'].ffill()
    #df_train['oil_price'] = df_train['oil_price'].bfill()
    df["onpromotion"] = df["onpromotion"].astype(np.int32)
    df['oil_price'] = df['oil_price'].astype(np.float32) 
    df['sales'] = df['sales'].astype(np.float32) 
    df['year']= df['year'].astype(np.int32)
    df['month']= df['month'].astype(np.int32)
    df['day_of_week']= df['day_of_week'].astype(np.int16)
    df['day_of_month'] = df['day_of_month'].astype(np.int16)
    df['weekofyear']= df['weekofyear'].astype(np.int16)
    df["family"] = df["family"].astype('category')
    df["onpromotion"] = df["onpromotion"].astype('category')
    df["type"] = df["type"].astype('category')
    df["cluster"] = df["cluster"].astype('category')
    df["holiday_type"] = df["holiday_type"].astype('category')
    df["is_holiday"] = df["holiday_type"].apply(lambda x: 1 if x=='Holiday' else 0)
    df["store_nbr"] = df["store_nbr"].astype('category')
    df["city"] = df["city"].astype('category')
    df["is_weekend"] = df["is_weekend"].astype('category')
    return df 




In [28]:
def add_rolling_data(df):
    df = db.sql("""
    with cte as (
    select *
    , avg(sales) OVER (PARTITION BY store_nbr, family, day_of_week ORDER BY "date" ASC ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING) AS "W3_rolling_mean_sales_store_family_dayofweek"
    from df 
    ) 
    select * 
    from cte -- where store_nbr==1 and family=='BEVERAGES'
    """).df()
    df = db.sql("""
    select *
    , avg(sales) OVER (PARTITION BY store_nbr, family ORDER BY "date" ASC ROWS BETWEEN 8 PRECEDING AND 1 PRECEDING) "D7_rolling_mean_sales_store_family"
    , avg(sales) OVER (PARTITION BY store_nbr, family ORDER BY "date" ASC ROWS BETWEEN 16 PRECEDING AND 1 PRECEDING) "15D_rolling_mean_sales_store_family"
    from df     
    """).df()
    df = db.sql("""
    select * 
    , lag(D7_rolling_mean_sales_store_family, 364) OVER (PARTITION BY store_nbr, family order by date asc) "lag1_7D_rolling_mean_sales_store_family" 
    , lag(D7_rolling_mean_sales_store_family, 182) OVER (PARTITION BY store_nbr, family order by date asc) "lag2_7D_rolling_mean_sales_store_family" 
    from df
    """).df() #
    return df

In [29]:
def add_seasonality_data(df):
    df_seasonality =db.sql("""
    with base as (
       select family, year, weekofyear, sum(sales) OVER (PARTITION BY family, year, weekofyear) "sales_qtrly" from df_train 
       where date < '2017-07-01' --and weekofyear <> 53 
    ), pivt as (
    PIVOT base
    ON year
    USING avg(sales_qtrly)    
    )
    select * from pivt 
    """).df().sort_values(['weekofyear']) #.query("store_nbr== 1 and family=='EGGS'")
    df_seasonality.loc[:,'Row_Total'] = df_seasonality[['2013','2014','2015','2016']].sum(numeric_only=True, axis=1)/4
    mean_mean_factor = np.mean(df_seasonality["Row_Total"])
    df_seasonality["scaled_seasonal_factor_weekly"] = df_seasonality["Row_Total"].apply(lambda x: x/mean_mean_factor)
    df = db.sql("""
        select t.*, dfs.scaled_seasonal_factor_weekly from df t 
        join df_seasonality dfs on t.family=dfs.family and t.weekofyear = dfs.weekofyear     
    """).df()#.query("store_nbr==1 and family=='EGGS'")#.to_clipboard()
    return df 

In [30]:
df_train = prep_train_data(pd.read_csv('data/train.csv'))
df_train = add_rolling_data(df_train)
df_train = add_seasonality_data(df_train)
df_train.columns

Index(['id', 'date', 'store_nbr', 'family', 'onpromotion', 'month', 'year',
       'day_of_week', 'day_of_month', 'quarter', 'weekofyear', 'payday',
       'is_weekend', 'sales', 'cluster', 'type', 'state', 'city',
       'holiday_type', 'oil_price', '7d_avg_oil_price', '90d_avg_oil_price',
       '15d_avg_oil_price', 'oil_lag364', '7d_transactions_avg',
       'number_of_holidays', 'is_national_holiday', 'earthquake_impact',
       'is_holiday', 'W3_rolling_mean_sales_store_family_dayofweek',
       'D7_rolling_mean_sales_store_family',
       '15D_rolling_mean_sales_store_family',
       'lag1_7D_rolling_mean_sales_store_family',
       'lag2_7D_rolling_mean_sales_store_family',
       'scaled_seasonal_factor_weekly'],
      dtype='object')

In [34]:
#prep test data 
entire_df = pd.concat([pd.read_csv('data/train.csv'), pd.read_csv('data/test.csv')])
entire_df
# test_df = prep_train_data(test_df)
# test_df = add_rolling_data(test_df)
# test_df = add_seasonality_data(test_df)
# test_df

,id,date,store_nbr,family,onpromotion,month,year,day_of_week,day_of_month,quarter,...,is_national_holiday,earthquake_impact,is_holiday,W3_rolling_mean_sales_store_family_dayofweek,D7_rolling_mean_sales_store_family,15D_rolling_mean_sales_store_family,lag1_7D_rolling_mean_sales_store_family,lag2_7D_rolling_mean_sales_store_family,scaled_seasonal_factor_weekly,future
0,2538136,2016-11-28 00:00:00,25,CLEANING,7,11.0,2016.0,0.0,28.0,4.0,...,0.0,0,0.0,486.25,358.50,368.3750,364.250,491.625,3.156493,NaN
1,2539918,2016-11-29 00:00:00,25,CLEANING,5,11.0,2016.0,1.0,29.0,4.0,...,0.0,0,0.0,495.00,333.50,351.6250,364.625,487.375,3.156493,NaN
2,2541700,2016-11-30 00:00:00,25,CLEANING,9,11.0,2016.0,2.0,30.0,4.0,...,0.0,0,0.0,800.00,321.25,336.7500,368.125,498.500,3.156493,NaN
3,2543482,2016-12-01 00:00:00,25,CLEANING,14,12.0,2016.0,3.0,1.0,4.0,...,0.0,0,0.0,594.75,326.00,335.7500,380.000,510.750,3.156493,NaN
4,2545264,2016-12-02 00:00:00,25,CLEANING,14,12.0,2016.0,4.0,2.0,4.0,...,0.0,0,0.0,492.75,328.50,340.9375,403.875,503.875,3.156493,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
28508,3029396,2017-08-31,9,PREPARED FOODS,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
28509,3029397,2017-08-31,9,PRODUCE,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y


In [ ]:
features = [ 'store_nbr', 
             'onpromotion' , 'family'
            #, 'cluster
            #, 'day_of_week'                       
            , 'W3_rolling_mean_sales_store_family_dayofweek'
            , 'D7_rolling_mean_sales_store_family' #, '15D_rolling_mean_sales_store_family' 
            , "7d_avg_oil_price" , "90d_avg_oil_price" 
            #, "15d_avg_oil_price"
            , "number_of_holidays"
            , "scaled_seasonal_factor_weekly" 
            , "oil_lag364"
            ,  "lag1_7D_rolling_mean_sales_store_family"
            , "lag2_7D_rolling_mean_sales_store_family"
    ]
TARGET = 'sales' 

X_train, y_train = df_train[features], df_train['sales']
#X_test, y_test = pjme_test[features], pjme_test['sales']
import sklearn.metrics
reg = XGBRegressor(n_estimators=20000, enable_categorical=True, early_stopping_rounds=15, eval_metric = 'rmsle', learning_rate = .3)
reg.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)])
pjme_test['prediction_label'] = reg.predict(X_test) 
pjme_test['prediction_label'] = pjme_test['prediction_label'].apply(lambda x: 0 if x<0 else x)
#pjme_test